In [67]:
import numpy as np

In [68]:
with open ("in.txt","r")as file:
    data=file.read().strip().split() 


In [69]:
A=float(data[0])
S=float(data[1])
N=float(data[2]) 

In [70]:
print(A)
print(S)
print(N)


0.6
1.0
100.0


In [71]:
clock=0.0
next_arrival_time=np.random.exponential(A)
next_departure_time=float("inf")
queue_length=0
surver_busy=False 

In [72]:
# Initialize additional variables
num_in_queue = 0
queue_arrival_times = []
server_busy = False

# Performance counters
num_delayed = 0       # number of customers who have started service
total_delay = 0.0     # sum of all delays in queue

# Time-average statistics
time_last_event = 0.0
area_num_in_q = 0.0
area_server_status = 0.0  # 1 when busy, 0 when idle

In [ ]:
# ---------- STEP 3: Open output file for event logs ----------
events_file = open("event_orders.txt", "w")

# Track customer numbers
customer_counter = 0
departing_customer_id = 0

# ---------- STEP 4: Main event loop ----------
event_counter = 0
while num_delayed < N:

    # 1) Identify next event
    if next_arrival_time <= next_departure_time:
        next_event_time = next_arrival_time
        next_event_type = "arrival"
    else:
        next_event_time = next_departure_time
        next_event_type = "departure"

    # 2) Update time-average statistics
    time_since_last_event = next_event_time - clock
    area_num_in_q += num_in_queue * time_since_last_event
    area_server_status += (1 if server_busy else 0) * time_since_last_event

    # Advance simulation time
    clock = next_event_time

    # 3) Process event
    if next_event_type == "arrival":
        event_counter += 1
        customer_counter += 1
        
        # Log arrival
        events_file.write(f"{event_counter}. Next event: Customer {customer_counter} Arrival\n")

        # Schedule next arrival
        next_arrival_time = clock + np.random.exponential(A)

        if not server_busy:
            # Server idle → customer gets service immediately
            server_busy = True
            num_delayed += 1  # this customer starts service now, no wait
            departing_customer_id = customer_counter

            # Generate service time and schedule departure
            service_time = np.random.exponential(S)
            next_departure_time = clock + service_time

            # No delay to record (delay = 0), so no delay log

        else:
            # Server busy → customer joins queue
            num_in_queue += 1
            queue_arrival_times.append(clock)  # store arrival time in queue

    else:  # departure event
        event_counter += 1
        
        # Log departure
        events_file.write(f"{event_counter}. Next event: Customer {departing_customer_id} Departure\n")

        # Write separator and counter after departures
        events_file.write(f"\n---------No. of customers delayed: {num_delayed}---------\n\n")

        if num_in_queue == 0:
            # No one is waiting in queue
            server_busy = False
            next_departure_time = float('inf')  # no further departure scheduled yet

        else:
            # Someone is waiting in the queue → take first customer
            num_in_queue -= 1
            arrival_time = queue_arrival_times.pop(0)

            # Calculate this customer's delay in queue
            delay = clock - arrival_time
            total_delay += delay

            # Now this customer starts service
            num_delayed += 1
            
            # Get the customer number from queue (need to track which customer is being served)
            # For this, we need to maintain customer IDs in queue
            departing_customer_id += 1

            # Schedule departure for this customer
            service_time = np.random.exponential(S)
            next_departure_time = clock + service_time

# Close events file
events_file.close()

# ---------- STEP 5: Compute performance measures ----------
avg_delay_in_queue = total_delay / num_delayed
avg_num_in_queue = area_num_in_q / clock
server_utilization = area_server_status / clock

# ---------- STEP 6: Write results to results.txt ----------
with open("results.txt", "w") as res:
    res.write(f"Average delay in queue: {avg_delay_in_queue:.6f}\n")
    res.write(f"Average number of customers in queue: {avg_num_in_queue:.6f}\n")
    res.write(f"Average server utilization: {server_utilization:.6f}\n")
    res.write(f"Time simulation ended: {clock:.6f}\n")
    res.write(f"Number of customers served: {num_delayed}\n")

print("Simulation finished. Check event_orders.txt and results.txt")

Simulation finished. Check event_orders.txt and results.txt
